# Install

Goal: At begining reduce and sequnce from n to b where b < n alllowing for savings of O(n^2-b^2) runtime

In [ ]:
!pip install huggingface_hub
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install datasets
!pip install wandb

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn
import torch.nn.functional as F
from huggingface_hub import login
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.auto import tqdm
from transformers import AutoTokenizer
import wandb
import os
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from datasets import load_dataset, Dataset
import shutil
from time import sleep
from datasets import load_dataset, DatasetDict, Dataset
from torch.nn.functional import binary_cross_entropy_with_logits
import random
from torch.nn.functional import binary_cross_entropy_with_logits
import random
login(token="ENTER TOKEN HERE")
wandb.login(key="ENTER KEY HERE")
device =  ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#If using collab
#from google.colab import drive
#drive.mount('/content/drive')

### **Helper functions**

In [ ]:
def create_random_forgetting_mask(input_ids, forget_prob):
      """
      Creates a mask for tokens to forget based on the forget rate.

      Args:
          input_ids (torch.Tensor): Tensor of shape (batch_size, seq_len).

      Returns:
          torch.Tensor: Binary mask of the same shape as input_ids,
                        where 1 indicates retained tokens and 0 indicates forgotten tokens.
      """
      batch_size, seq_len = input_ids.size()
      num_tokens_to_forget = int((seq_len) * forget_prob)
      masks = []

      for i in range(batch_size):
          # Generate random indices for tokens to forget
          forget_indices = torch.randperm(seq_len)[:num_tokens_to_forget]
          mask = torch.ones(seq_len, dtype=torch.bool, device=input_ids.device)
          mask[forget_indices] = False  # Mark forgotten tokens as 0
          masks.append(mask)

      return torch.stack(masks)

In [ ]:
def apply_mask( input_ids, mask, padding_value=0):
      """
      Applies a forgetting mask to input_ids to retain only unmasked tokens and pads sequences to uniform length.

      Args:
          input_ids (torch.Tensor): Tensor of shape (batch_size, seq_len).
          mask (torch.Tensor): Binary mask of the same shape as input_ids,
                              where 1 indicates retained tokens and 0 indicates forgotten tokens.
          padding_value (int, optional): Value to pad the sequences to uniform length. Defaults to 0.

      Returns:
          torch.Tensor: Padded tensor of shape (batch_size, max_len), where max_len is the length of the longest retained sequence.
      """
      updated_input_ids = []

      for i in range(input_ids.size(0)):
          # Use the mask to retain tokens from input_ids
          retained_input_ids = input_ids[i][mask[i].bool()]
          updated_input_ids.append(retained_input_ids)


      # Find the maximum sequence length after masking
      max_length = max(ids.size(0) for ids in updated_input_ids)

      if max_length == 0:
        #only return last tokens
        return input_ids[:, -1]


        # Pad sequences to the maximum length
      padded_input_ids = torch.full(
            (input_ids.size(0), max_length),
            padding_value,
            dtype=input_ids.dtype,
            device=input_ids.device
        )

      for i, ids in enumerate(updated_input_ids):
          padded_input_ids[i, :ids.size(0)] = ids

      return padded_input_ids


In [ ]:
def memory_check_and_empty():
    """Check GPU memory and clear cache only if necessary."""
    if torch.cuda.is_available():
        allocated_memory = torch.cuda.memory_allocated()
        reserved_memory = torch.cuda.memory_reserved()
        if reserved_memory - allocated_memory > 0.1 * reserved_memory:  # Threshold for unused memory
            torch.cuda.empty_cache()

In [ ]:
def get_labels(mask, input_ids)

### **Models**

Models create Masks for input tokens

In [ ]:
class IdentityMask(nn.Module):
  def __init__(self):
      super(IdentityMask, self).__init__()

  def forward(self, input_ids):
      return torch.ones_like(input_ids, dtype=torch.bool)


In [ ]:
class RandomForgetMask(nn.Module):
  def __init__(self, forget_prob = .1):
      super(RandomForgetMask, self).__init__()
      self.forget_prob = forget_prob

  def create_random_forgetting_mask(self, input_ids):
      """
      Creates a mask for tokens to forget based on the forget rate.

      Args:
          input_ids (torch.Tensor): Tensor of shape (batch_size, seq_len).

      Returns:
          torch.Tensor: Binary mask of the same shape as input_ids,
                        where 1 indicates retained tokens and 0 indicates forgotten tokens.
      """
      batch_size, seq_len = input_ids.size()
      num_tokens_to_forget = int((seq_len) * self.forget_prob)
      masks = []

      for i in range(batch_size):
          # Generate random indices for tokens to forget
          forget_indices = torch.randperm(seq_len)[:num_tokens_to_forget]
          mask = torch.ones(seq_len, dtype=torch.bool, device=input_ids.device)
          mask[forget_indices] = False  # Mark forgotten tokens as 0
          masks.append(mask)

      return torch.stack(masks)

  def forward(self, input_ids):
      mask = self.create_random_forgetting_mask(input_ids)
      return mask


In [ ]:
class MLPForgetMask(nn.Module):
    def __init__(self, base_llm_model, hidden_dim=512):
        """
        Initializes the MLPForgetMask module.

        Args:
            base_llm_model (nn.Module): The base language model (e.g., a transformer model).
            tokenizer (PreTrainedTokenizer): The tokenizer corresponding to the language model.
            threshold (float): Threshold for forgetting. Probabilities below this are set to 0 (forgotten).
            hidden_dim (int): Hidden layer size for the MLP.
        """
        super(MLPForgetMask, self).__init__()
        self.embedding = base_llm_model.model.embed_tokens

        # Retrieve the embedding dimension from the base model
        embedding_dim = base_llm_model.config.hidden_size

        # Define the MLP
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # Outputs a probability for retaining each token
        )

    def create_mlp_forgetting_mask(self, tokens):
        """
        Creates a mask for tokens to retain/forget using the MLP.

        Args:
            tokens (torch.Tensor): Token embeddings of shape (batch_size, seq_len, embedding_dim).

        Returns:
            torch.Tensor: Binary mask of shape (batch_size, seq_len),
                          where 1 indicates retained tokens and 0 indicates forgotten tokens.
        """
        batch_size, seq_len, embed_dim = tokens.size()

        # Flatten token embeddings for processing through MLP
        tokens_flat = tokens.view(-1, embed_dim)  # Shape: (batch_size * seq_len, embedding_dim)

        # Compute retain probabilities
        retain_probs = self.mlp(tokens_flat).view(batch_size, seq_len)  # Shape: (batch_size, seq_len)

        return retain_probs

    def forward(self, input_ids):
        """
        Generates the forgetting mask and applies it to the token embeddings.

        Args:
            input_ids (torch.Tensor): Input token IDs of shape (batch_size, seq_len).

        Returns:
            torch.Tensor: Masked token embeddings.
        """
        # Get token embeddings
        tokens = self.embedding(input_ids)  # Shape: (batch_size, seq_len, embedding_dim)

        # Create mask
        mask = self.create_mlp_forgetting_mask(tokens)  # Shape: (batch_size, seq_len)

        return mask


In [ ]:
class CNNMLPForgetMask(nn.Module):
    def __init__(self, base_llm_model, hidden_dim=512, cnn_hidden_dim=256, kernel_size=3):
        """
        Initializes the CNNMLPForgetMask module.

        Args:
            base_llm_model (nn.Module): The base language model (e.g., a transformer model).
            threshold (float): Threshold for forgetting. Probabilities below this are set to 0 (forgotten).
            hidden_dim (int): Hidden layer size for the MLP.
            cnn_hidden_dim (int): Number of output channels for the CNN.
            kernel_size (int): Size of the convolutional kernel.
        """
        super(CNNMLPForgetMask, self).__init__()
        self.embedding = base_llm_model.model.embed_tokens

        # Retrieve the embedding dimension from the base model
        embedding_dim = base_llm_model.config.hidden_size

        # Define the CNN layer
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=cnn_hidden_dim, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.ReLU(),
            nn.Conv1d(in_channels=cnn_hidden_dim, out_channels=embedding_dim, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.ReLU()
        )

        # Define the MLP
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # Outputs a probability for retaining each token
        )

    def create_cnn_mlp_forgetting_mask(self, tokens):
        """
        Creates a mask for tokens to retain/forget using CNN + MLP.

        Args:
            tokens (torch.Tensor): Token embeddings of shape (batch_size, seq_len, embedding_dim).

        Returns:
            torch.Tensor: Binary mask of shape (batch_size, seq_len),
                          where 1 indicates retained tokens and 0 indicates forgotten tokens.
        """
        batch_size, seq_len, embed_dim = tokens.size()

        # Apply CNN: Convert to (batch_size, embed_dim, seq_len) for Conv1d
        tokens_cnn_input = tokens.transpose(1, 2)  # Shape: (batch_size, embed_dim, seq_len)
        tokens_cnn_output = self.cnn(tokens_cnn_input)  # Shape: (batch_size, embed_dim, seq_len)
        tokens_processed = tokens_cnn_output.transpose(1, 2)  # Shape: (batch_size, seq_len, embed_dim)

        # Flatten token embeddings for processing through MLP
        tokens_flat = tokens_processed.view(-1, embed_dim)  # Shape: (batch_size * seq_len, embed_dim)

        # Compute retain probabilities
        retain_probs = self.mlp(tokens_flat).view(batch_size, seq_len)  # Shape: (batch_size, seq_len)


        return retain_probs

    def forward(self, input_ids):
        """
        Generates the forgetting mask and applies it to the token embeddings.

        Args:
            input_ids (torch.Tensor): Input token IDs of shape (batch_size, seq_len).

        Returns:
            torch.Tensor: Masked token embeddings.
        """
        # Get token embeddings
        tokens = self.embedding(input_ids)  # Shape: (batch_size, seq_len, embedding_dim)

        # Create mask using CNN + MLP
        mask = self.create_cnn_mlp_forgetting_mask(tokens)  # Shape: (batch_size, seq_len)

        return mask

In [ ]:
class RNNForgetMask(nn.Module):
    def __init__(self, base_llm_model, threshold=0.5, hidden_dim=512, rnn_type="GRU", bidirectional=True):
        """
        Initializes the RNNForgetMask module.

        Args:
            base_llm_model (nn.Module): The base language model (e.g., a transformer model).
            threshold (float): Threshold for forgetting. Probabilities below this are set to 0 (forgotten).
            hidden_dim (int): Hidden layer size for the RNN.
            rnn_type (str): Type of RNN to use ("RNN", "LSTM", or "GRU").
        """
        super(RNNForgetMask, self).__init__()
        self.embedding = base_llm_model.model.embed_tokens
        self.threshold = threshold

        # Retrieve the embedding dimension from the base model
        embedding_dim = base_llm_model.config.hidden_size

        # Define the RNN layer
        rnn_class = {"RNN": nn.RNN, "LSTM": nn.LSTM, "GRU": nn.GRU}[rnn_type]
        self.rnn = rnn_class(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=bidirectional)

        if bidirectional:
            hidden_dim *= 2

        # Linear layer to output probabilities
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # Outputs a probability for retaining each token
        )

    def create_rnn_forgetting_mask(self, tokens):
        """
        Creates a mask for tokens to retain/forget using the RNN.

        Args:
            tokens (torch.Tensor): Token embeddings of shape (batch_size, seq_len, embedding_dim).

        Returns:
            torch.Tensor: Binary mask of shape (batch_size, seq_len),
                          where 1 indicates retained tokens and 0 indicates forgotten tokens.
        """
        batch_size, seq_len, embed_dim = tokens.size()
       # print('dfdfsfdsdfdfsfds')
       # print(tokens.shape)

        # Pass tokens through RNN
        rnn_output, _ = self.rnn(tokens)  # rnn_output: (batch_size, seq_len, hidden_dim)

      #  print(rnn_output.shape)

        # Compute retain probabilities
        retain_probs = self.output_layer(rnn_output).squeeze(-1)  # Shape: (batch_size, seq_len)


        return retain_probs

    def forward(self, input_ids):
        """
        Generates the forgetting mask and applies it to the token embeddings.

        Args:
            input_ids (torch.Tensor): Input token IDs of shape (batch_size, seq_len).

        Returns:
            torch.Tensor: Masked token embeddings.
        """
        # Get token embeddings
        tokens = self.embedding(input_ids)  # Shape: (batch_size, seq_len, embedding_dim)

        # Create mask
        mask = self.create_rnn_forgetting_mask(tokens)  # Shape: (batch_size, seq_len)

        return mask

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token

### **Training**

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
batch_size = 4
EXAMPLES = 10000

In [ ]:
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")
train_data = dataset["train"].select(range(EXAMPLES))
# Initialize tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Function to preprocess examples
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenized_dataset = train_data.map(preprocess_function, batched=True, remove_columns=['text'])
tokenized_dataset.set_format(type='torch', columns=['input_ids'])

train_dataloader = DataLoader(tokenized_dataset, shuffle=True, batch_size=batch_size, num_workers=min(4, torch.get_num_threads()),
        pin_memory=True,
        prefetch_factor=2,  # Prefetch batches
        persistent_workers=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_llm_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B").to(device)

In [ ]:
#Chose Mask Type
#mask = IdentityMask().to(device)
#mask = RandomForgetMask(forget_prob=.1).to(device)
#mask = MLPForgetMask(base_llm_model).to(device)
#mask = RNNForgetMask(base_llm_model, bidirectional=False).to(device)

soft = False
parameters = list(base_llm_model.parameters()) + list(mask.parameters())
name = "ENTER NAME HERE"
num_epochs = 5

# Optimizer and Scheduler
optimizer = AdamW(parameters, lr=5e-5)
total_steps = num_epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

In [ ]:

def reinforce_model(base_model, forget_layer, forget_p,exploration_epochs,mask_batch, train_dataloader, optimizer, scheduler, num_epochs, device, model_name, save_dir,  batch_size=64):

    wandb.init(project="forget_language_model_training", config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "learning_rate": optimizer.param_groups[0]['lr'],
        "model_name": model_name,
        "dataset": "wikipedia",
    })

    save_dir = os.path.join(save_dir, model_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    accumulation_steps = batch_size // train_dataloader.batch_size  # Gradient accumulation steps
    total_steps = num_epochs * len(train_dataloader) // accumulation_steps
    save_interval = (total_steps // 3) +1
    step_count = 0
    progress_bar = tqdm(total=total_steps, desc="Training")

    # Initialize mixed precision scaler
    scaler = GradScaler()

    base_model.train()
    forget_layer.train()
    for epoch in range(num_epochs):
        epoch_loss = 0  # Reset loss for each epoch
        for batch_idx, batch in enumerate(train_dataloader):
          input_ids = batch["input_ids"].to(device)
          explore = (random.random() <= exploration_epochs[epoch])

          if explore:
            random_mask = create_random_forgetting_mask(input_ids, forget_p).to(device)
          else:
            predicted_mask_logits = forget_layer(input_ids)

          if explore:
            mask = random_mask
          else:
            k = int(predicted_mask_logits.size(1) * (1-forget_p))
            _, indices = torch.topk(predicted_mask_logits, k, dim=1)
            new_mask = torch.zeros_like(predicted_mask_logits)
            new_mask.scatter_(1, indices, 1)  # Set top-k values to 1
            mask = new_mask


          new_input_ids = apply_mask(input_ids, mask)
          batchsize, seq_len = new_input_ids.size()
          labels = input_ids[:, 1:]
          labels = apply_mask(torch.cat([labels, torch.zeros((labels.size(0), 1), dtype=torch.long)], dim=1))

          with autocast():
            outputs = base_llm_model(input_ids=new_input_ids,  labels=labels)#attention_mask=attention_mask,
            loss = outputs.loss / accumulation_steps
            mloss = loss
            if explore:
              if True:
                with torch.no_grad():
                  input_ids = batch["input_ids"].to(device)
                  mask_data = []
                  for _ in range(mask_batch//train_dataloader.batch_size):
                    random_mask = create_random_forgetting_mask(input_ids, forget_p).to(device)
                    new_input_ids = apply_mask(input_ids, random_mask)
                    batchsize, seq_len = new_input_ids.size()
                    labels = input_ids[:, 1:]
                    labels = apply_mask(torch.cat([labels, torch.zeros((labels.size(0), 1), dtype=torch.long)], dim=1))
                    outputs = base_llm_model(input_ids=new_input_ids,  labels=labels)
                    mask_data.append([random_mask, outputs.loss.item()])
                predicted_mask_logits = forget_layer(input_ids)
                random_masks = torch.stack([item[0] for item in mask_data]).to(device)
                losses = torch.tensor([item[1] for item in mask_data]).to(device)

                loss_weights = torch.exp(-losses)  # Exponential decay to emphasize lower losses
                normalized_weights = loss_weights / torch.sum(loss_weights)

                expanded_predicted_mask_logits =  predicted_mask_logits.unsqueeze(0).repeat_interleave(mask_batch//train_dataloader.batch_size, dim=0)
                bce_loss = binary_cross_entropy_with_logits(
                    expanded_predicted_mask_logits, random_masks.float(), reduction="none"
                )
                normalized_weights = normalized_weights.unsqueeze(1).repeat(1, 4)
                mask_loss = (normalized_weights.unsqueeze(-1) * bce_loss).mean()/ accumulation_steps

                mloss = loss + mask_loss
            else:
              mloss = loss

          scaler.scale(mloss).backward()

          if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_dataloader):
              # Gradient update
              scaler.step(optimizer)
              scheduler.step()

              scaler.update()

              optimizer.zero_grad()

              step_count += 1
              progress_bar.update(1)
              logs = {"step_loss": loss.item() * accumulation_steps, "step": step_count}
              if explore:
                logs["reinforce"] = mask_loss.item() *accumulation_steps
              wandb.log(logs)

          epoch_loss += loss.item() * accumulation_steps  # Accumulate scaled loss
          memory_check_and_empty()
        # Epoch logging
        avg_epoch_loss = epoch_loss / len(train_dataloader)
        wandb.log({"epoch_loss": avg_epoch_loss, "epoch": epoch})

    # Final model save
    save_path_forget = os.path.join(save_dir, f"forget.pt")
    base_model.save_pretrained(save_dir)
    torch.save(forget_layer, save_path_forget)
    print(f"Model saved")
    wandb.finish()


In [ ]:
#Rienforcement Learning with given list of probability to explore or not each batch during a epoch
#forget_p = .10
#mask_batch = 256
#exploration_epochs = [.75, .50,.25, 0]
#assert len(exploration_epochs) == num_epochs

In [ ]:
#reinforce_model(base_llm_model, mask,forget_p, exploration_epochs,mask_batch,  train_dataloader,  optimizer, scheduler, num_epochs, device, name, "/content/drive/My Drive/llama/models")

In [ ]:
def two_step_reinforce_model(base_model, forget_layer, forget_p,exploration_epochs,mask_batch, train_dataloader, optimizer, scheduler, num_epochs, device, model_name, save_dir,  batch_size=64):

    wandb.init(project="forget_language_model_training", config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "learning_rate": optimizer.param_groups[0]['lr'],
        "model_name": model_name,
        "dataset": "wikipedia",
    })

    save_dir = os.path.join(save_dir, model_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    accumulation_steps = batch_size // train_dataloader.batch_size  # Gradient accumulation steps
    total_steps = num_epochs * len(train_dataloader) // accumulation_steps
    save_interval = (total_steps // 3) +1
    step_count = 0
    progress_bar = tqdm(total=total_steps, desc="Training")

    # Initialize mixed precision scaler
    scaler = GradScaler()

    base_model.train()
    forget_layer.train()
    main_epoch = 0
    for epoch in range(num_epochs):
        epoch_loss = 0  # Reset loss for each epoch
        for batch_idx, batch in enumerate(train_dataloader):
          input_ids = batch["input_ids"].to(device)
          explore = exploration_epochs[epoch]

          if explore:
            with torch.no_grad():
              input_ids = batch["input_ids"].to(device)
              mask_data = []
              for _ in range(mask_batch//train_dataloader.batch_size):
                random_mask = create_random_forgetting_mask(input_ids, forget_p).to(device)
                new_input_ids = apply_mask(input_ids, random_mask)
                batchsize, seq_len = new_input_ids.size()
                labels = input_ids[:, 1:]
                labels = apply_mask(torch.cat([labels, torch.zeros((labels.size(0), 1), dtype=torch.long)], dim=1))
                outputs = base_llm_model(input_ids=new_input_ids,  labels=labels)
                mask_data.append([random_mask, outputs.loss.item()])
            predicted_mask_logits = forget_layer(input_ids)
            random_masks = torch.stack([item[0] for item in mask_data]).to(device)
            losses = torch.tensor([item[1] for item in mask_data]).to(device)

            loss_weights = torch.exp(-losses)  # Exponential decay to emphasize lower losses
            normalized_weights = loss_weights / torch.sum(loss_weights)

            expanded_predicted_mask_logits =  predicted_mask_logits.unsqueeze(0).repeat_interleave(mask_batch//train_dataloader.batch_size, dim=0)
            bce_loss = binary_cross_entropy_with_logits(
                    expanded_predicted_mask_logits, random_masks.float(), reduction="none"
                )
            normalized_weights = normalized_weights.unsqueeze(1).repeat(1, 4)
            loss = (normalized_weights.unsqueeze(-1) * bce_loss).sum()/ accumulation_steps

          else:
            predicted_mask_logits = forget_layer(input_ids)
            k = int(predicted_mask_logits.size(1) * (1-forget_p))
            _, indices = torch.topk(predicted_mask_logits, k, dim=1)
            new_mask = torch.zeros_like(predicted_mask_logits)
            new_mask.scatter_(1, indices, 1)  # Set top-k values to 1
            mask = new_mask

            new_input_ids = apply_mask(input_ids, mask)
            batchsize, seq_len = new_input_ids.size()
            labels = input_ids[:, 1:]
            labels = apply_mask(torch.cat([labels, torch.zeros((labels.size(0), 1), dtype=torch.long)], dim=1))

            with autocast():
              outputs = base_llm_model(input_ids=new_input_ids,  labels=labels)#attention_mask=attention_mask,
              loss = outputs.loss / accumulation_steps

          scaler.scale(loss).backward()

          if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_dataloader):
              # Gradient update
              scaler.step(optimizer)

              scaler.update()

              optimizer.zero_grad()

              step_count += 1
              progress_bar.update(1)
              logs = { "step": step_count}
              if explore:
                logs["reinforce"] = loss.item() *accumulation_steps
              else:
                scheduler.step()
                logs["step_loss"] = loss.item() *accumulation_steps
              wandb.log(logs)

          memory_check_and_empty()
          if not explore:
            epoch_loss += loss.item() * accumulation_steps  # Accumulate scaled loss

        # Epoch logging
        if not explore:
          avg_epoch_loss = epoch_loss / len(train_dataloader)
          wandb.log({"epoch_loss": avg_epoch_loss, "epoch": main_epoch})
          main_epoch +=1

    # Final model save
    save_path_forget = os.path.join(save_dir, f"forget.pt")
    base_model.save_pretrained(save_dir)
    torch.save(forget_layer, save_path_forget)
    print(f"Model saved")
    wandb.finish()

In [ ]:
#Rienforcement Learning with given list of explore or not at each epoch
forget_p = .10
mask_batch = 16
exploration_epochs = [True, False ,True, False, False]
assert len(exploration_epochs) == num_epochs

In [ ]:
two_step_reinforce_model(base_llm_model, mask,forget_p, exploration_epochs,mask_batch,  train_dataloader,  optimizer, scheduler, num_epochs, device, name, "/content/drive/My Drive/llama/models")